In [1]:
!pip install duckdb --upgrade
import sys
sys.exit(0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 49.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.2.0
    Uninstalling duckdb-1.2.0:
      Successfully uninstalled duckdb-1.2.0
sys.exit called with value 0. The interpreter will be restarted.


**<mark>write ducklake</mark>**

In [8]:
schema_name = "helloworld"

In [9]:
%%time
import duckdb
token = notebookutils.credentials.getToken('storage')
con = duckdb.connect()
con.sql(f"""
  CREATE or replace SECRET onelake ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}') ;
  ATTACH or replace 'ducklake:/lakehouse/default/Files/{schema_name}.db' AS lake
          (DATA_PATH 'az://onelake.blob.fabric.microsoft.com/tmp/data.Lakehouse/Tables') ;
  USE lake ;
  CALL set_option('parquet_row_group_size', 2048*1000) ;
  CALL set_option('target_file_size', '128MB') ;
  CALL set_option('parquet_compression', 'ZSTD');
  CALL set_option('parquet_version', 2);
  """)

CPU times: user 649 ms, sys: 39.9 ms, total: 688 ms
Wall time: 433 ms


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [10]:
con.sql(f"""
              create schema if not exists {schema_name} ;
              use {schema_name} ;
              create  table if not exists summary as select * from
               delta_scan('abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/spark_unsorted/summary')
               order by date,DUID,time;
 """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
for x in ['calendar','mstdatetime','duid']:
   con.sql(f""" create  table if not exists {x} as select * from
    delta_scan('abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/spark_unsorted/{x}');
    """)

In [12]:
con.sql(f"""
  checkpoint __ducklake_metadata_lake
  """)
con.close()


In [13]:
!pip install -q  duckrun
import duckrun
con = duckrun.connect(f'tmp/data.lakehouse/dbo')
con.export_ducklake_to_delta(f"{schema_name}.db")

🔌 Attaching tables from schema: dbo
🔍 Discovering tables via OneLake Delta Table API...
   Using identifier: tmp/data.Lakehouse
   Listing tables in schema: dbo
   Found 5 tables
results, resultss, sales_partitioned, test, resultsss
🔍 Exporting DuckLake metadata from: helloworld.db
📂 Full DB path: abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/helloworld.db
Processing helloworld.duid...
✅ Exported DuckLake snapshot 5 as Delta checkpoint v5
✅ Uploaded to: abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/helloworld/duid/_delta_log/
Processing helloworld.mstdatetime...
✅ Exported DuckLake snapshot 4 as Delta checkpoint v4
✅ Uploaded to: abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/helloworld/mstdatetime/_delta_log/
Processing helloworld.calendar...
✅ Exported DuckLake snapshot 3 as Delta checkpoint v3
✅ Uploaded to: abfss://tmp@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/helloworld/calendar/_delta_log/
Processing hellow

True